In [1]:
import os
import pandas as pd
import numpy as np
import re
from IPython.display import clear_output

In [2]:
merged_dataset=pd.DataFrame()
counter=0
num_to_process=len(os.listdir('datasets'))
num_files=0
num_sheets=0
for filename in os.listdir('datasets'):
    # Exclude "filtered" files, txt files, and Excel hidden files
    if 'Filtered' not in filename and 'xlsx' in filename and '$' not in filename:
        num_files+=1
        sheets=pd.ExcelFile('datasets/'+filename,engine='openpyxl')
        # Extract predictors from filename
        split_filename=filename.split(',')
        Fe203_flowrate=int(split_filename[0].split('Fe')[0])
        ITCMO_support_fraction=int(split_filename[1].split('Support')[0])
        pressure=int(split_filename[2].split('atm')[0].strip())
        factor_compressor=None
        if 'Comp' in filename:
            factor_compressor=int(re.findall('([0-9])Comp',filename)[0])
        # Iterate through each sheet in Excel file
        for sheet in sheets.sheet_names:
            print(filename)
            print(sheet)
            # Check if sheetname contains temperature value (should contain "C")
            if 'C' not in sheet:
                temp='ERROR'
            else:
                # Extract temperature variable
                temp=int(sheet.split('C')[0])
            num_sheets+=1
            # Read in current sheet
            dataset=pd.read_excel('datasets/'+filename,engine='openpyxl',sheet_name=sheet,index_col='Row/Case')
            # Clean column names
            dataset.columns=[' '.join(i.replace('_x000D_\n','').split()) for i in dataset.columns]
            # Add temperature variable from sheetname to dataset
            dataset['temperature']=temp
            # Add predictors from filename to dataset
            dataset['Fe203_flowrate']=Fe203_flowrate
            dataset['ITCMO_support_fraction']=ITCMO_support_fraction
            dataset['pressure']=pressure
            if factor_compressor!=None:
                dataset['factor_compressor']=factor_compressor
            # Drop columns with no name (empty)
            dataset.drop(columns=[column for column in dataset.columns if 'Unnamed' in column],inplace=True)
            # Merge into master dataset
            merged_dataset=pd.concat([merged_dataset,dataset])
    clear_output(wait=True)
    counter+=1
    print('Merged '+str(counter)+' of '+str(num_to_process)+'!')

Merged 67 of 67!


In [3]:
print('Num files: '+str(num_files))
print('Num sheets: '+str(num_sheets))

Num files: 37
Num sheets: 148


In [4]:
merged_dataset.columns

Index(['Status', 'VARY 1CO2IN MIXED TOTAL MOLEFLOW KMOL/HR',
       'VARY 2H2OIN MIXED TOTAL MOLEFLOW KMOL/HR', 'CH4 KMOL/HR', 'CO KMOL/HR',
       'CO2 KMOL/HR', 'H2 KMOL/HR', 'H2O KMOL/HR', 'FE2O3 KMOL/HR',
       'FE3O4 KMOL/HR', 'FEO KMOL/HR', 'FETIO3 KMOL/HR', 'FE KMOL/HR',
       'C KMOL/HR', 'REDT C', 'COMBT C', 'Q1 MW', 'Q2 MW', 'Q3 MW', 'Q4 MW',
       'COM1 MW', 'COM2 MW', 'COM3 MW', 'COM4 MW', 'COM5 MW', 'P1 ATM',
       'P2 ATM', 'P3 ATM', 'P4 ATM', 'AIR KMOL/HR', 'temperature',
       'Fe203_flowrate', 'ITCMO_support_fraction', 'pressure', 'Q6 MW',
       'P5 ATM', 'factor_compressor', 'Q5 MW', 'COM6 MW', 'P6 ATM', 'P5',
       'PUMP MW'],
      dtype='object')

In [5]:
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550856 entries, 1 to 3722
Data columns (total 42 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Status                                    550856 non-null  object 
 1   VARY 1CO2IN MIXED TOTAL MOLEFLOW KMOL/HR  550856 non-null  int64  
 2   VARY 2H2OIN MIXED TOTAL MOLEFLOW KMOL/HR  550856 non-null  int64  
 3   CH4 KMOL/HR                               550856 non-null  float64
 4   CO KMOL/HR                                550856 non-null  float64
 5   CO2 KMOL/HR                               550856 non-null  float64
 6   H2 KMOL/HR                                550856 non-null  float64
 7   H2O KMOL/HR                               550856 non-null  float64
 8   FE2O3 KMOL/HR                             550856 non-null  int64  
 9   FE3O4 KMOL/HR                             550856 non-null  float64
 10  FEO KMOL/HR           

In [2]:
import pandas as pd
merged=pd.read_csv('merged_dataset.csv')

In [4]:
merged[['Fe203_flowrate','FE2O3 KMOL/HR']].describe()

,Fe203_flowrate,FE2O3 KMOL/HR
count,550856.000000,550856.0
mean,43648.648649,0.0
std,10309.323478,0.0
min,30000.000000,0.0
25%,30000.000000,0.0
50%,45000.000000,0.0
75%,55000.000000,0.0
max,55000.000000,0.0


In [6]:
merged_dataset.isnull().mean()

Status                                      0.000000
VARY 1CO2IN MIXED TOTAL MOLEFLOW KMOL/HR    0.000000
VARY 2H2OIN MIXED TOTAL MOLEFLOW KMOL/HR    0.000000
CH4 KMOL/HR                                 0.000000
CO KMOL/HR                                  0.000000
CO2 KMOL/HR                                 0.000000
H2 KMOL/HR                                  0.000000
H2O KMOL/HR                                 0.000000
FE2O3 KMOL/HR                               0.000000
FE3O4 KMOL/HR                               0.000000
FEO KMOL/HR                                 0.000000
FETIO3 KMOL/HR                              0.000000
FE KMOL/HR                                  0.000000
C KMOL/HR                                   0.000000
REDT C                                      0.000000
COMBT C                                     0.000000
Q1 MW                                       0.000000
Q2 MW                                       0.000000
Q3 MW                                       0.

In [7]:
merged_dataset.isnull().mean()[merged_dataset.isnull().mean()>0].index

Index(['AIR KMOL/HR', 'Q6 MW', 'P5 ATM', 'factor_compressor', 'Q5 MW',
       'COM6 MW', 'P6 ATM', 'P5', 'PUMP MW'],
      dtype='object')

In [8]:
merged_dataset.describe()

,VARY 1CO2IN MIXED TOTAL MOLEFLOW KMOL/HR,VARY 2H2OIN MIXED TOTAL MOLEFLOW KMOL/HR,CH4 KMOL/HR,CO KMOL/HR,CO2 KMOL/HR,H2 KMOL/HR,H2O KMOL/HR,FE2O3 KMOL/HR,FE3O4 KMOL/HR,FEO KMOL/HR,...,ITCMO_support_fraction,pressure,Q6 MW,P5 ATM,factor_compressor,Q5 MW,COM6 MW,P6 ATM,P5,PUMP MW
count,550856.000000,550856.000000,550856.000000,550856.000000,550856.000000,550856.000000,550856.000000,550856.0,550856.000000,550856.000000,...,550856.000000,550856.000000,167446.000000,163768.000000,133992.000000,297760.000000,44664.000000,4.466400e+04,14888.0,3722.000000
mean,29994.626545,29994.626545,1583.094324,19037.337523,34564.869604,28491.329687,49562.727857,0.0,3.013691,2.577854,...,68.000000,15.351351,485.677716,9.090967,3.333333,-68.177014,40.814412,1.000031e+01,0.0,0.670655
std,17607.518692,17607.518692,2583.824214,6701.145881,15183.155011,7873.054762,16559.045668,0.0,5.285587,10.096457,...,14.684073,10.511821,275.637216,1.484451,0.471406,18.198389,9.676913,4.899034e-07,0.0,0.380753
min,0.000000,0.000000,0.003244,2969.530790,2068.685230,8800.912770,5844.056500,0.0,0.000000,0.000000,...,50.000000,1.000000,15.475579,6.666876,3.000000,-106.166967,28.256131,1.000031e+01,0.0,0.021659
25%,15000.000000,15000.000000,14.967142,14129.860575,21614.940925,22771.810400,36512.180125,0.0,0.000000,0.000000,...,50.000000,10.000000,250.989967,6.666876,3.000000,-80.700066,28.256131,1.000031e+01,0.0,0.346541
50%,30000.000000,30000.000000,210.178686,18457.703150,33737.620500,27403.472550,49020.907500,0.0,0.000000,0.000000,...,66.000000,10.000000,486.293060,10.000001,3.000000,-71.616051,42.384197,1.000031e+01,0.0,0.671423
75%,45000.000000,45000.000000,1968.393487,23281.622575,46834.287450,33132.224025,62338.250225,0.0,8.187403,0.000000,...,85.000000,20.000000,721.596154,10.000001,4.000000,-51.728625,51.802907,1.000031e+01,0.0,0.996305
max,60000.000000,60000.000000,12504.492900,44705.932000,72324.340000,57498.363800,91610.900900,0.0,15.010233,47.550898,...,85.000000,30.000000,941.212375,10.000002,4.000000,-28.055158,51.802907,1.000031e+01,0.0,1.299528


In [9]:
merged_dataset.shape

(550856, 42)

In [10]:
merged_dataset.to_csv('merged_dataset.csv')